In [65]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [66]:
orders_url_public = 'https://drive.google.com/file/d/1715E6ynv_sSgDuelOIB8g0DMdyUZRHj8/view?usp=share_link'
orders_url_processed = 'https://drive.google.com/uc?id=' + orders_url_public.split('/')[-2]
df_orders = pd.read_csv(orders_url_processed)

#nodes_url_public = 'https://drive.google.com/file/d/1VIqyXcoUg0fxEzieu01t5pOTSpsip6AE/view?usp=share_link'
#nodes_url_processed = 'https://drive.google.com/uc?id=' + nodes_url_public.split('/')[-2]
#df_nodes = pd.read_csv(nodes_url_processed)


In [67]:
df_orders.head()

,Unnamed: 0,Id,running_time,completed_time,route_distance_km,delta_time,is_afternoon,is_evening,is_morning,is_night,...,center_cluster_20,center_cluster_21,center_cluster_22,center_cluster_23,center_cluster_24,center_cluster_25,center_cluster_26,center_cluster_27,center_cluster_28,center_cluster_29
0,0,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,0,1,0,0,...,0.000187,0.000089,0.000352,0.000223,0.000361,0.003389,0.001009,0.965341,0.000146,0.000343
1,1,-7575630690398473489,2022-01-24 06:53:53,2022-01-24 07:06:26,3.526,753.0,0,0,1,0,...,0.000727,0.000474,0.001438,0.000926,0.001176,0.006062,0.002630,0.028300,0.000603,0.002082
2,2,-6264582368520213833,2022-01-24 10:00:59,2022-01-24 10:15:58,5.071,899.0,0,0,1,0,...,0.002052,0.001067,0.003604,0.002392,0.003860,0.025585,0.009700,0.232100,0.001619,0.003763
3,3,5964315354301636538,2022-01-24 14:28:05,2022-01-24 14:35:08,2.867,423.0,1,0,0,0,...,0.000781,0.000653,0.000958,0.000814,0.001092,0.001724,0.001445,0.002135,0.000686,0.001092
4,4,1372379574816145639,2022-01-24 11:57:29,2022-01-24 12:06:29,3.751,540.0,0,0,1,0,...,0.001406,0.002246,0.003671,0.002517,0.001294,0.002202,0.001769,0.002392,0.001492,0.896434


In [68]:
df_nodes = pd.read_csv('drive/MyDrive/hackathon/df_prepared_rnn_v3.csv')

In [69]:
df_nodes.head()

,Unnamed: 0,Id,distance,speed,lat_s,lon_s,lat_f,lon_f,hour,route_distance_km,...,morning_peak_time,evening_peak_time,temperature_2m (°C),relativehumidity_2m (%),apparent_temperature (°C),snowfall (cm),dewpoint_2m (°C),cloudcover (%),windspeed_10m (km/h),windgusts_10m (km/h)
0,0,-6374252502568484586,17.414917,25,46.472665,30.739248,46.472682,30.739022,10,2.156,...,0,0,-2.8,73,-10.4,0.07,-7.0,100,32.1,52.6
1,1,-6374252502568484586,17.186843,27,46.472507,30.741320,46.472524,30.741097,10,2.156,...,0,0,-2.8,73,-10.4,0.07,-7.0,100,32.1,52.6
2,2,-6374252502568484586,25.777868,32,46.472813,30.737195,46.472835,30.736860,10,2.156,...,0,0,-2.8,73,-10.4,0.07,-7.0,100,32.1,52.6
3,3,-6374252502568484586,2.690220,33,46.467890,30.736439,46.488425,30.711088,10,2.156,...,0,0,-2.8,73,-10.4,0.07,-7.0,100,32.1,52.6
4,4,-6374252502568484586,83.792561,35,46.473025,30.734244,46.473103,30.733156,10,2.156,...,0,0,-2.8,73,-10.4,0.07,-7.0,100,32.1,52.6


In [70]:
df_nodes.columns

Index(['Unnamed: 0', 'Id', 'distance', 'speed', 'lat_s', 'lon_s', 'lat_f',
       'lon_f', 'hour', 'route_distance_km', 'delta_time', 'is_afternoon',
       'is_evening', 'is_morning', 'is_night', 'morning_peak_time',
       'evening_peak_time', 'temperature_2m (°C)', 'relativehumidity_2m (%)',
       'apparent_temperature (°C)', 'snowfall (cm)', 'dewpoint_2m (°C)',
       'cloudcover (%)', 'windspeed_10m (km/h)', 'windgusts_10m (km/h)'],
      dtype='object')

In [71]:
stat_params = ['speed','distance']
for col in stat_params:
  df_orders = pd.merge(df_orders, df_nodes.groupby('Id')[col].mean().rename(f'{col}_mean'),on='Id')
  df_orders = pd.merge(df_orders, df_nodes.groupby('Id')[col].min().rename(f'{col}_min'),on='Id')
  df_orders = pd.merge(df_orders, df_nodes.groupby('Id')[col].max().rename(f'{col}_max'),on='Id')
  df_orders = pd.merge(df_orders, df_nodes.groupby('Id')[col].std().rename(f'{col}_std'),on='Id')


In [72]:
cluster_cols = []
for col in df_orders.columns:
  if 'cluster' in col:
    cluster_cols.append(col)

In [73]:
df_orders.drop(columns=['Unnamed: 0','Id','completed_time'],inplace=True)

In [79]:
df_orders.drop(columns=cluster_cols,inplace=True)

In [75]:
df_orders['running_time'] = pd.to_datetime(df_orders['running_time'])

In [80]:
df_orders.drop(columns=['temperature_2m (°C)', 'relativehumidity_2m (%)', 'apparent_temperature (°C)', 'snowfall (cm)', 'dewpoint_2m (°C)', 'cloudcover (%)', 'windspeed_10m (km/h)', 'windgusts_10m (km/h)'],inplace=True)



In [81]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_orders.drop(columns=['delta_time']), df_orders['delta_time'], test_size=0.07, random_state=42)

In [77]:
!pip install catboost


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [82]:
import catboost as cb

model = cb.CatBoostRegressor()

model.fit(X_train,y_train,eval_set=(X_test,y_test))


Learning rate set to 0.06481
0:	learn: 206.9093191	test: 208.7801081	best: 208.7801081 (0)	total: 6ms	remaining: 5.99s
1:	learn: 200.6593809	test: 202.3234276	best: 202.3234276 (1)	total: 11.6ms	remaining: 5.78s
2:	learn: 194.5308896	test: 196.1785653	best: 196.1785653 (2)	total: 16.8ms	remaining: 5.58s
3:	learn: 189.0980562	test: 190.5245226	best: 190.5245226 (3)	total: 21.9ms	remaining: 5.45s
4:	learn: 183.9845782	test: 185.5461787	best: 185.5461787 (4)	total: 26.9ms	remaining: 5.35s
5:	learn: 179.1912609	test: 181.2467177	best: 181.2467177 (5)	total: 32ms	remaining: 5.3s
6:	learn: 175.0694938	test: 177.3291660	best: 177.3291660 (6)	total: 37.4ms	remaining: 5.3s
7:	learn: 171.2228837	test: 173.4575535	best: 173.4575535 (7)	total: 42.4ms	remaining: 5.25s
8:	learn: 167.9795674	test: 170.1559524	best: 170.1559524 (8)	total: 47.4ms	remaining: 5.22s
9:	learn: 165.0581119	test: 167.2341818	best: 167.2341818 (9)	total: 52.6ms	remaining: 5.21s
10:	learn: 161.9187488	test: 163.9349012	best: 1

In [19]:
speed_mean = df_nodes.groupby('Id')['speed'].mean().rename('speed_mean')

In [20]:
speed_mean.head()

Id
-9220688251826578095    25.901235
-9218029690111550526    29.971963
-9203204462098983368    33.301370
-9183832189489697758    35.923729
-9182793427449706037    24.270270
Name: speed_mean, dtype: float64

In [13]:
speed_std = df_nodes.groupby('Id')['speed'].std()

In [15]:
pd.concat([df_orders, speed_mean], axis=0, join="inner")

""
0
1
2
3
4
...
9210081165830935160
9210110364218727646
9210371713303339999
9211624789772099086


In [18]:
temp = pd.merge(df_orders, speed_mean, on="Id")
temp[temp.Id==-9220688251826578095]['speed']

2465    25.901235
Name: speed, dtype: float64

In [7]:
coord_cols = ['lat_s','lon_s','lat_f','lon_f']
new_cols = []
for col in coord_cols:
  new_col_name = f'{col}_label'
  df_nodes[new_col_name]=pd.cut(df_nodes[col],30,labels=list(range(30)))
  new_cols.append(new_col_name)

In [8]:
temp_df = pd.get_dummies(df_nodes[new_cols])
df_nodes[temp_df.columns] = temp_df

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


In [9]:
df_nodes

,Unnamed: 0,Id,distance,speed,lat_s,lon_s,lat_f,lon_f,hour,route_distance_km,...,lon_f_label_20,lon_f_label_21,lon_f_label_22,lon_f_label_23,lon_f_label_24,lon_f_label_25,lon_f_label_26,lon_f_label_27,lon_f_label_28,lon_f_label_29
0,0,-6374252502568484586,-0.459246,-0.610289,0.405096,0.316816,0.405678,0.306050,10,-1.174222,...,0,0,0,0,0,0,0,0,0,0
1,1,-6374252502568484586,-0.464588,-0.412809,0.400435,0.415847,0.401040,0.405170,10,-1.174222,...,0,0,0,0,0,0,0,0,0,0
2,2,-6374252502568484586,-0.263368,0.080890,0.409461,0.218678,0.410204,0.202716,10,-1.174222,...,0,0,0,0,0,0,0,0,0,0
3,3,-6374252502568484586,-0.804129,0.179630,0.264072,0.182552,0.870469,-1.028710,10,-1.174222,...,0,0,0,0,0,0,0,0,0,0
4,4,-6374252502568484586,1.095458,0.377110,0.415728,0.077612,0.418110,0.025767,10,-1.174222,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368320,368320,6454990024190504891,-0.239542,-0.807768,0.286036,-5.453848,0.288075,-5.384307,18,-1.154589,...,0,0,0,0,0,0,0,0,0,0
368321,368321,6454990024190504891,1.537872,-0.215329,0.328210,-5.177367,0.355603,-5.177213,18,-1.154589,...,0,0,0,0,0,0,0,0,0,0
368322,368322,6454990024190504891,1.016491,-0.215329,0.382954,-5.182123,0.382180,-5.229583,18,-1.154589,...,0,0,0,0,0,0,0,0,0,0
368323,368323,6454990024190504891,2.265227,-0.215329,0.381401,-5.270021,0.380049,-5.350186,18,-1.154589,...,0,0,0,0,0,0,0,0,0,0


In [10]:
'day' in df_nodes.columns

False

In [11]:
df_nodes.head()

,Unnamed: 0,Id,distance,speed,lat_s,lon_s,lat_f,lon_f,hour,route_distance_km,...,lon_f_label_20,lon_f_label_21,lon_f_label_22,lon_f_label_23,lon_f_label_24,lon_f_label_25,lon_f_label_26,lon_f_label_27,lon_f_label_28,lon_f_label_29
0,0,-6374252502568484586,-0.459246,-0.610289,0.405096,0.316816,0.405678,0.306050,10,-1.174222,...,0,0,0,0,0,0,0,0,0,0
1,1,-6374252502568484586,-0.464588,-0.412809,0.400435,0.415847,0.401040,0.405170,10,-1.174222,...,0,0,0,0,0,0,0,0,0,0
2,2,-6374252502568484586,-0.263368,0.080890,0.409461,0.218678,0.410204,0.202716,10,-1.174222,...,0,0,0,0,0,0,0,0,0,0
3,3,-6374252502568484586,-0.804129,0.179630,0.264072,0.182552,0.870469,-1.028710,10,-1.174222,...,0,0,0,0,0,0,0,0,0,0
4,4,-6374252502568484586,1.095458,0.377110,0.415728,0.077612,0.418110,0.025767,10,-1.174222,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df_nodes.drop(columns=['Unnamed: 0'],inplace=True)

In [13]:
features = list(set(df_nodes.columns) - set(['Id','delta_time','lat_s','lon_s','lat_f','lon_f']))

In [14]:

# req_one_hot_cols = set(df_target.columns)-set(['userid','canceled','event_created_date'])

#one_hot_df = pd.get_dummies(df_target[list(req_one_hot_cols)],dummy_na=True)
X = []
Y = []
for id in df_nodes['Id'].unique():
  cur_rows_df_target = df_nodes[df_nodes['Id']==id]
  y = df_nodes[df_nodes['Id']==id].iloc[0]['delta_time']
  X.append(cur_rows_df_target[features].values)
  Y.append(y)

X = tf.keras.utils.pad_sequences(X)
Y = np.array(Y).astype(float)

In [15]:
X.shape

(4997, 264, 142)

In [16]:
Y.shape

(4997,)

In [17]:
Y[:5]

array([436., 710., 771., 394., 347.])

In [18]:
X = tf.cast(X, tf.float32)
Y = tf.cast(Y, tf.float32)

In [29]:
from keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))


model = tf.keras.Sequential([tf.keras.layers.SimpleRNN(192,)
                                                  #kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4),
                                                  #bias_regularizer=tf.keras.regularizers.L2(1e-4),
                                                  #activity_regularizer=tf.keras.regularizers.L2(1e-5)),
                             #tf.keras.layers.Dropout(0.1),
                             ,tf.keras.layers.Dense(1)])

model.compile(loss=root_mean_squared_error,
              optimizer=tf.keras.optimizers.Adam())

In [30]:
history = model.fit(X,Y,epochs=90,batch_size=64,validation_split=0.2)

Epoch 1/90
63/63 [==============================] - 66s 193ms/step - loss: 650.2335 - val_loss: 541.9493
Epoch 2/90
63/63 [==============================] - 13s 207ms/step - loss: 635.9993 - val_loss: 530.3223
Epoch 3/90
63/63 [==============================] - 12s 189ms/step - loss: 624.4398 - val_loss: 518.9323
Epoch 4/90
63/63 [==============================] - 11s 178ms/step - loss: 612.7047 - val_loss: 507.6761
Epoch 5/90
63/63 [==============================] - 11s 172ms/step - loss: 601.1703 - val_loss: 496.5231
Epoch 6/90
63/63 [==============================] - 11s 183ms/step - loss: 589.7282 - val_loss: 485.4622
Epoch 7/90
63/63 [==============================] - 12s 185ms/step - loss: 578.1841 - val_loss: 474.4853
Epoch 8/90
63/63 [==============================] - 12s 186ms/step - loss: 567.1860 - val_loss: 463.5948
Epoch 9/90
63/63 [==============================] - 12s 186ms/step - loss: 555.6307 - val_loss: 452.7968
Epoch 10/90
63/63 [==============================] - 11

In [24]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model rmse')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: ignored

In [26]:
model.save('drive/MyDrive/hackathon/model_onehot_features_v2.h5')